In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [7]:
import getpass
import os

if not os.environ.get("Groq_API_Key"):
    os.environ["GROQ_API_KEY"]=getpass.getpass("Enter API key for Groq:")

In [8]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [ ]:
RESPONSE_JSON = {
    "1": {
        "Q1": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
    "2": {
        "Q2": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
    "3": {
        "Q3": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
}

In [ ]:
template = """
Text:{text}
You are an expert AI MCQ maker. From the above given text, it is you job to \
create a quiz of {number} multiple choice questions for {subject} students in a {difficult} level.
Make Sure the questions do not get repeated and check all the questions to be vomforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

# can write this as well,
# template = """
# You are an AI teacher. Based on the following text, create 3 multiple-choice questions (MCQs) with the correct answers. 
# The questions should be relevant and based on the key concepts in the text. The format should be as follows:
# 1. Question: {question}
#    a) Option 1
#    b) Option 2
#    c) Option 3
#    d) Option 4
#    Correct answer: {correct_answer}

# Text: {text}

# Please generate the MCQs.
# """

In [ ]:
Quiz_Generation_Template = PromptTemplate(
    input_variables=["text", "number", "subject", "level", "response_json"],
    template=template
)